LSTM模型
===
LSTM(Long Short-Term Memory)，也称长短时记忆结构，它是传统RNN的变体，与经典RNN相比能够有效捕捉长序列之间的语义关联，环节梯度消失或爆炸现象，同时LSTM的结构更复杂，它的核心机构可以分为四个部分去解析：遗忘门、输入门、细胞状态、输出门。

# 1.LSTM模型的整体架构

![images](images/013.png)

![images](images/014.png)

对于中间的方框内部来说，可以分为三个部分，最左边的就是遗忘门，中间的就是输入门，最右边的就是输出门

# 2.遗忘门

![images](images/015.png)

$$f_t=\sigma[W_f \bullet (h_{t-1}, x_t) + b_f]$$

与传统RNN的内部结构计算非常相似，首先将当前时间步输入$x_t$与上一个时间步隐含状态$h_{t-1}$拼接，得到$(x_t, h_{t-1})$，然后通过一个全连接层做变换，最后通过sigmoid函数进行激活得到$f_t$，我们可以将$f_t$看作是门值，好比一扇门开合的大小程度，门值豆浆作用在通过该扇门的张量上，遗忘门门值将作用的上一层的细胞状态上，代表遗忘过去的信息多少，有因为遗忘门门值是由$(x_t, h_{t-1})$计算得来的，因此整个公式意味着根据当前时间步输入和上一个时间步隐含状态$h_{t-1}$来决定遗忘多少上一层的细胞状态所携带的过往信息。$\sigma$就是sigmoid激活函数

# 4.输入门

![images](images/016.png)

$$
\begin{cases}
i_t &=& \sigma[W_i \bullet (h_{t-1},x_t) + b_i] \\
\tilde{C_t} &=& tanh[W_C \bullet (h_{t-1},x_t) + b_C]
\end{cases}
$$

## 4.1.输入门结构分析
输入门的计算公式有两个，第一个就是产生输入门门值的公式，它和遗忘门公式几乎相同，区别只是在于它们之后要作用的目标上，这个公式意味着输入信息有多少需要进行过滤。输入门的第二个公式是与传统RNN的内部结构计算相同。对于LSTM来讲，它得到的是当前的细胞状态，儿不是像经典RNN一样得到的是隐含状态

## 4.2.细胞状态更新图与计算公式

![images](images/017.png)

$C_t=f_t \times C_{t-1} + i_t \times \tilde{C_t}$

细胞更新的结构与计算公式非常容易理解，这里没有全连接层，只是将刚刚得到的遗忘门门值与上一个时间步得到的$C_{t-1}$相乘，在加上输入门门值与当前时间步得到的未更新$C_t$相乘的结果，最终得到更新后的$C_t$作为下一个时间步输入的一部分。整个细胞状态更新过程就是对遗忘门和输入门的应用。

# 5.输出门

![images](images/018.png)

$$
\begin{split}
o_t&=&\sigma[W_o \bullet (h_{t-1},x_t)+b_o] \\
h_t&=&o_t \times tanh(C_t)
\end{split}
$$

输出门部分的公式也是两个，第一个即是计算输出门的门值，它和遗忘门，输入门计算方式相同。第二个即是使用这个门值产生隐含状态$h_t$，它将作用在更新后的细胞状态$C_t$上，并做tanh激活，最终得到$h_t$作为下一时间步输入的一部分，整个输出门的过程，就是为了产生隐含状态$h_t$

# 6.Bi-LSTM(双向LSTM)
Bi-LSTM即双向LSTM，它没有改变LSTM本身任何的内部结构，只是将LSTM应用两次且方向不通，再将两次得到的LSTM结果进行拼接作为最终输出。

![images](images/019.png)

上图中“我爱中国”这个输入序列进行了从左到右和从右到左两次LSTM处理，将得到的结果张量进行了拼接作为最终输出。这种结构能够捕捉语言语法中一些特定的前置和后置特征，增强语义关联，但是模型参数和计算复杂度也随之增加一倍，一半需要对预料和计算资源进行评估后决定是否使用该结构

# 7.Pytorch中LSTM的实现

In [1]:
import torch.nn as nn
import torch
rnn = nn.LSTM(5, 6, 2)
input = torch.randn(1, 3, 5)
h0 = torch.randn(2, 3, 6)
c0 = torch.randn(2, 3, 6)
output, (hn, cn) = rnn(input, (h0, c0))
output

tensor([[[-3.1438e-01, -1.2928e-01, -3.7069e-02, -1.9658e-01,  1.0967e-01,
           7.9839e-02],
         [-3.9546e-01,  1.9518e-01, -4.8389e-03,  3.0838e-01,  6.1793e-02,
          -3.8651e-01],
         [-2.5879e-01,  1.7589e-01,  1.5065e-01,  1.9216e-04,  1.6868e-01,
          -9.9255e-02]]], grad_fn=<StackBackward>)

In [2]:
hn

tensor([[[-7.2975e-03, -5.5939e-02,  3.2803e-02, -5.2007e-02, -5.7420e-01,
           1.7728e-01],
         [ 2.4220e-01,  1.5650e-01,  9.1452e-02,  2.4841e-01, -2.8458e-01,
          -1.1954e-01],
         [ 9.7167e-02,  2.8160e-02, -2.7460e-01,  7.8425e-01,  9.2205e-02,
          -5.1066e-01]],

        [[-3.1438e-01, -1.2928e-01, -3.7069e-02, -1.9658e-01,  1.0967e-01,
           7.9839e-02],
         [-3.9546e-01,  1.9518e-01, -4.8389e-03,  3.0838e-01,  6.1793e-02,
          -3.8651e-01],
         [-2.5879e-01,  1.7589e-01,  1.5065e-01,  1.9216e-04,  1.6868e-01,
          -9.9255e-02]]], grad_fn=<StackBackward>)

In [3]:
cn

tensor([[[-2.1829e-02, -8.5236e-02,  1.2193e-01, -1.0253e-01, -1.0826e+00,
           3.4727e-01],
         [ 5.1476e-01,  4.8721e-01,  2.1957e-01,  5.1965e-01, -3.7373e-01,
          -3.4252e-01],
         [ 1.1754e-01,  1.0520e-01, -7.3106e-01,  1.9698e+00,  1.2075e-01,
          -1.2936e+00]],

        [[-7.6427e-01, -3.2770e-01, -3.0148e-01, -7.4928e-01,  2.3884e-01,
           2.8404e-01],
         [-7.7367e-01,  2.5603e-01, -1.1134e-02,  5.9767e-01,  2.3559e-01,
          -1.3907e+00],
         [-7.7103e-01,  2.4428e-01,  2.9180e-01,  2.7269e-04,  6.8762e-01,
          -1.5894e-01]]], grad_fn=<StackBackward>)

# 8.LSTM的优势和劣势

## 8.1.LSTM的优势
LSTM的门结构能够有效减缓长序列问题中可能出现的梯度消失或梯度爆炸，虽然并不能杜绝这种现象，但在更长序列问题上表现优于传统RNN

## 8.2.LSTM的缺点
优于内部结构相对比较复杂，因此训练效率在同等算力下较传统RNN低很多